In [2]:
import pandas as pd
from func import cal_buoyancy,sounding_cal
import glob
import os
from haversine import haversine, Unit
import re
import datetime
import metpy.calc as mpcalc
from metpy.units import units
from datetime import datetime, timedelta
from metpy import constants
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from scipy.interpolate import interp1d
from scipy.optimize import bisect
import numpy as np
import pickle
import numpy as np
import math
import pint
import seaborn as sns
import tropycal.tracks as tracks
from pydoc import help
from sklearn.preprocessing import QuantileTransformer
import xarray as xr
import netCDF4 as nc
import re
import matplotlib.pyplot as plt
from numpy import load
from numpy import asarray
from numpy import save
import pytz
from scipy import stats
from sklearn.linear_model import LinearRegression
from collections import Counter
import pymannkendall as mk
from io import StringIO 
import requests
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

In [3]:
# Set display options to show all rows and columns in dataframe
pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)

In [4]:
hurdat = tracks.TrackDataset(basin='north_atlantic',source='hurdat',include_btk=False)

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (2.93 seconds)


In [8]:
#read in precipitation files:
#2001-2019 warm-season

directory_path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/abrfc_final"

#files either .cdf or .nc
#file name looks like: sgpabrfcprecipX1.c1.19971231.000000.custom.cdf
#file name looks like: sgpabrfcprecipX1.c1.20171231.000000.custom.nc

files = [
    
    os.path.join(directory_path, file)
    
    for file in os.listdir(directory_path)
    
    if file.endswith(".cdf") or file.endswith(".nc")
]

filtered_files = []

for file in files:
    
    file_name = os.path.basename(file)
    
    year_month = file_name.split('.')[2]
    
    if len(year_month) == 8:
        
        year = int(year_month[0:4])
        
        month = int(year_month[4:6])
        
        day = int(year_month[6:8])
        
        hours_mins_seconds = file_name.split('.')[3]
        
        hour = int(hours_mins_seconds[0:2])
        
        if 5 <= month <= 9 and 2001 <= year <= 2018:
            
            filtered_files.append(file)
    

precipdfs = []

for ds in sorted(filtered_files):
    
    ds = xr.open_dataset(ds)
    
    precipdfs.append(ds)
    
len(precipdfs)

2753

In [7]:
# annie -- hourly precipitation calculation

cleanp = []

for df in precipdfs:
    
    min_p = 0
    
    #equivalent to 250mm of rain per hour
    
    max_p = 25000

    precip = df.variables['precip'].data
          
    mask = ((precip < min_p) | (precip > max_p))

    precip_clean = np.where(mask, np.nan, precip)
    
    nan_count = np.count_nonzero(np.isnan(precip_clean))

    if nan_count == 0:
        
        cleanp.append(df)
        

print(len(cleanp))

2898


In [11]:
sgp = (36.607322, -97.487643)

max_distance = 100.0  # kilometers

precip_pairs = []

precip_index_of_pairs = []

for i in range(160):

    for j in range(336):

        lat = cleanp[0].variables["lat"].data[i, j]

        lon = cleanp[0].variables["lon"].data[i, j]

        lat_lon_pair = (lat,lon)

        distance = haversine(sgp, lat_lon_pair, unit='km')

        if distance <= max_distance:

            pair = (lat, lon)

            index = (i, j)

            precip_pairs.append(pair)

            precip_index_of_pairs.append(index)

len(precip_index_of_pairs)

1894

In [22]:
hourlyp = []

count=0

for df in cleanp:
    
    hrs = df.variables['precip'].shape[0]
    
    for hr in range(hrs):
        
        ptotal = 0
        
        time = df['time_offset'].data[hr]
        
        lst = time - pd.to_timedelta('6 hours')
    
        for index in precip_index_of_pairs:
          
            p = df.variables['precip'].data[hr,index[0],index[1]]
            
            #convert to mm
            
            p = p * 0.01
                
            ptotal += p
    
        avgp = ptotal/len(precip_index_of_pairs)
    
        tup = (lst, avgp)
        
        hourlyp.append(tup)
    
    count+=1
    print((count/2898)*100, "% complete")

0.03450655624568668 % complete
0.06901311249137336 % complete
0.10351966873706005 % complete
0.13802622498274672 % complete
0.1725327812284334 % complete
0.2070393374741201 % complete
0.24154589371980675 % complete
0.27605244996549344 % complete
0.3105590062111801 % complete
0.3450655624568668 % complete
0.3795721187025535 % complete
0.4140786749482402 % complete
0.4485852311939268 % complete
0.4830917874396135 % complete
0.5175983436853002 % complete
0.5521048999309869 % complete
0.5866114561766735 % complete
0.6211180124223602 % complete
0.6556245686680469 % complete
0.6901311249137336 % complete
0.7246376811594203 % complete
0.759144237405107 % complete
0.7936507936507936 % complete
0.8281573498964804 % complete
0.862663906142167 % complete
0.8971704623878536 % complete
0.9316770186335404 % complete
0.966183574879227 % complete
1.000690131124914 % complete
1.0351966873706004 % complete
1.069703243616287 % complete
1.1042097998619738 % complete
1.1387163561076603 % complete
1.1732229

9.696342305037957 % complete
9.730848861283643 % complete
9.765355417529332 % complete
9.799861973775018 % complete
9.834368530020704 % complete
9.86887508626639 % complete
9.903381642512077 % complete
9.937888198757763 % complete
9.972394755003451 % complete
10.006901311249138 % complete
10.041407867494824 % complete
10.07591442374051 % complete
10.110420979986197 % complete
10.144927536231885 % complete
10.179434092477571 % complete
10.213940648723257 % complete
10.248447204968944 % complete
10.28295376121463 % complete
10.317460317460316 % complete
10.351966873706004 % complete
10.38647342995169 % complete
10.420979986197377 % complete
10.455486542443063 % complete
10.48999309868875 % complete
10.524499654934436 % complete
10.559006211180124 % complete
10.59351276742581 % complete
10.628019323671497 % complete
10.662525879917183 % complete
10.697032436162871 % complete
10.731538992408558 % complete
10.766045548654244 % complete
10.800552104899932 % complete
10.835058661145618 % comp

19.25465838509317 % complete
19.289164941338854 % complete
19.32367149758454 % complete
19.358178053830226 % complete
19.392684610075914 % complete
19.4271911663216 % complete
19.461697722567287 % complete
19.496204278812975 % complete
19.530710835058663 % complete
19.565217391304348 % complete
19.599723947550036 % complete
19.63423050379572 % complete
19.66873706004141 % complete
19.703243616287093 % complete
19.73775017253278 % complete
19.772256728778466 % complete
19.806763285024154 % complete
19.841269841269842 % complete
19.875776397515526 % complete
19.910282953761214 % complete
19.944789510006903 % complete
19.979296066252587 % complete
20.013802622498275 % complete
20.048309178743963 % complete
20.082815734989648 % complete
20.117322291235336 % complete
20.15182884748102 % complete
20.18633540372671 % complete
20.220841959972393 % complete
20.25534851621808 % complete
20.28985507246377 % complete
20.324361628709458 % complete
20.358868184955142 % complete
20.39337474120083 % c

28.81297446514838 % complete
28.847481021394067 % complete
28.88198757763975 % complete
28.91649413388544 % complete
28.951000690131124 % complete
28.985507246376812 % complete
29.020013802622497 % complete
29.054520358868185 % complete
29.08902691511387 % complete
29.123533471359558 % complete
29.158040027605246 % complete
29.19254658385093 % complete
29.227053140096622 % complete
29.261559696342303 % complete
29.296066252587995 % complete
29.330572808833676 % complete
29.365079365079367 % complete
29.39958592132505 % complete
29.43409247757074 % complete
29.468599033816425 % complete
29.503105590062113 % complete
29.5376121463078 % complete
29.572118702553485 % complete
29.606625258799173 % complete
29.641131815044858 % complete
29.675638371290546 % complete
29.71014492753623 % complete
29.74465148378192 % complete
29.779158040027603 % complete
29.81366459627329 % complete
29.848171152518976 % complete
29.882677708764664 % complete
29.91718426501035 % complete
29.951690821256037 % co

38.44030365769496 % complete
38.47481021394065 % complete
38.50931677018634 % complete
38.54382332643202 % complete
38.57832988267771 % complete
38.612836438923395 % complete
38.64734299516908 % complete
38.68184955141477 % complete
38.71635610766045 % complete
38.75086266390614 % complete
38.78536922015183 % complete
38.81987577639752 % complete
38.8543823326432 % complete
38.88888888888889 % complete
38.923395445134574 % complete
38.95790200138026 % complete
38.99240855762595 % complete
39.02691511387164 % complete
39.061421670117326 % complete
39.09592822636301 % complete
39.130434782608695 % complete
39.16494133885438 % complete
39.19944789510007 % complete
39.23395445134575 % complete
39.26846100759144 % complete
39.30296756383713 % complete
39.33747412008282 % complete
39.371980676328505 % complete
39.406487232574186 % complete
39.44099378881988 % complete
39.47550034506556 % complete
39.51000690131125 % complete
39.54451345755693 % complete
39.57902001380263 % complete
39.613526

...

57.86749482401656 % complete
57.90200138026225 % complete
57.936507936507944 % complete
57.971014492753625 % complete
58.005521048999306 % complete
58.040027605244994 % complete
58.07453416149069 % complete
58.10904071773637 % complete
58.14354727398205 % complete
58.17805383022774 % complete
58.212560386473434 % complete
58.247066942719115 % complete
58.2815734989648 % complete
58.31608005521049 % complete
58.35058661145618 % complete
58.38509316770186 % complete
58.41959972394755 % complete
58.454106280193244 % complete
58.488612836438925 % complete
58.523119392684606 % complete
58.557625948930294 % complete
58.59213250517599 % complete
58.62663906142167 % complete
58.66114561766735 % complete
58.69565217391305 % complete
58.730158730158735 % complete
58.764665286404416 % complete
58.7991718426501 % complete
58.83367839889579 % complete
58.86818495514148 % complete
58.90269151138716 % complete
58.93719806763285 % complete
58.97170462387854 % complete
59.006211180124225 % complete
59.

67.59834368530021 % complete
67.6328502415459 % complete
67.66735679779158 % complete
67.70186335403726 % complete
67.73636991028296 % complete
67.77087646652863 % complete
67.80538302277432 % complete
67.83988957902002 % complete
67.8743961352657 % complete
67.90890269151139 % complete
67.94340924775707 % complete
67.97791580400276 % complete
68.01242236024845 % complete
68.04692891649412 % complete
68.08143547273981 % complete
68.11594202898551 % complete
68.15044858523119 % complete
68.18495514147688 % complete
68.21946169772258 % complete
68.25396825396825 % complete
68.28847481021394 % complete
68.32298136645963 % complete
68.35748792270532 % complete
68.391994478951 % complete
68.42650103519668 % complete
68.46100759144237 % complete
68.49551414768807 % complete
68.53002070393374 % complete
68.56452726017943 % complete
68.59903381642512 % complete
68.63354037267081 % complete
68.6680469289165 % complete
68.70255348516217 % complete
68.73706004140787 % complete
68.77156659765356 %

77.39820565907523 % complete
77.4327122153209 % complete
77.46721877156659 % complete
77.50172532781228 % complete
77.53623188405797 % complete
77.57073844030366 % complete
77.60524499654935 % complete
77.63975155279503 % complete
77.67425810904072 % complete
77.7087646652864 % complete
77.7432712215321 % complete
77.77777777777779 % complete
77.81228433402346 % complete
77.84679089026915 % complete
77.88129744651484 % complete
77.91580400276052 % complete
77.95031055900621 % complete
77.9848171152519 % complete
78.01932367149759 % complete
78.05383022774328 % complete
78.08833678398895 % complete
78.12284334023465 % complete
78.15734989648033 % complete
78.19185645272601 % complete
78.2263630089717 % complete
78.26086956521739 % complete
78.29537612146308 % complete
78.32988267770877 % complete
78.36438923395445 % complete
78.39889579020014 % complete
78.43340234644583 % complete
78.4679089026915 % complete
78.50241545893721 % complete
78.53692201518288 % complete
78.57142857142857 % 

87.23257418909593 % complete
87.26708074534162 % complete
87.3015873015873 % complete
87.33609385783299 % complete
87.37060041407867 % complete
87.40510697032437 % complete
87.43961352657004 % complete
87.47412008281573 % complete
87.50862663906143 % complete
87.5431331953071 % complete
87.5776397515528 % complete
87.61214630779848 % complete
87.64665286404417 % complete
87.68115942028986 % complete
87.71566597653553 % complete
87.75017253278122 % complete
87.78467908902692 % complete
87.8191856452726 % complete
87.85369220151829 % complete
87.88819875776397 % complete
87.92270531400966 % complete
87.95721187025535 % complete
87.99171842650104 % complete
88.02622498274673 % complete
88.06073153899241 % complete
88.09523809523809 % complete
88.12974465148378 % complete
88.16425120772948 % complete
88.19875776397515 % complete
88.23326432022084 % complete
88.26777087646653 % complete
88.30227743271222 % complete
88.3367839889579 % complete
88.37129054520358 % complete
88.40579710144928 %

97.06694271911664 % complete
97.10144927536231 % complete
97.135955831608 % complete
97.17046238785369 % complete
97.20496894409938 % complete
97.23947550034507 % complete
97.27398205659075 % complete
97.30848861283644 % complete
97.34299516908213 % complete
97.3775017253278 % complete
97.4120082815735 % complete
97.4465148378192 % complete
97.48102139406487 % complete
97.51552795031056 % complete
97.55003450655624 % complete
97.58454106280193 % complete
97.61904761904762 % complete
97.65355417529331 % complete
97.688060731539 % complete
97.72256728778468 % complete
97.75707384403036 % complete
97.79158040027606 % complete
97.82608695652173 % complete
97.86059351276742 % complete
97.89510006901311 % complete
97.9296066252588 % complete
97.96411318150449 % complete
97.99861973775018 % complete
98.03312629399586 % complete
98.06763285024155 % complete
98.10213940648723 % complete
98.13664596273291 % complete
98.17115251897862 % complete
98.20565907522429 % complete
98.24016563146998 % co

In [167]:
pdf = pd.DataFrame(hourlyp, columns=['time','p'])

pdf['timestamp'] = pd.to_datetime(pdf['time'])

pdf['date'] = pdf['timestamp'].dt.date

res = pdf.groupby('date')['p'].agg(list).reset_index()

res = res.set_index('date')

res.head()

,p
date,
2001-04-30,"[0.0, 0.0, 0.0, 0.004213305174234423, 0.001166..."
2001-05-01,"[0.00020591341077085535, 0.0, 0.0, 0.0, 0.0, 0..."
2001-05-02,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2001-05-03,"[0.11404963041182672, 0.6078299894403384, 0.41..."
2001-05-04,"[0.014445617740232312, 0.025564941921858493, 0..."


In [168]:
tuples = []

for i in range(res.shape[0]):
    
    date = res.index[i]
    
    precip = np.array(res['p'][i])
    
    if len(precip) == 24:
        
        #corresponds to hours 6-13
        precip_morning = sum(precip[7:15])
        
        #corresponds to hours 14-20
        precip_afternoon = sum(precip[15:22]) 
    
        #corresponds to hours 21-24
        precip_evening = sum(precip[22:24])+precip[0]
        
        peak_hr = np.argmax(precip)
        
        peak_amount = precip[peak_hr]
        
        tup = (date, precip_morning, precip_afternoon, precip_evening, peak_hr, peak_amount)
        
        tuples.append(tup)

aperes = pd.DataFrame(tuples,columns=['date','morning','afternoon','evening','peak_hr', 'peak_amount'])

In [169]:
conditions = (aperes['afternoon'] > aperes['evening']) & \
             (aperes['afternoon'] > 2 * aperes['morning']) & \
             (aperes['peak_hr'].isin([15, 16, 17, 18, 19, 20, 21]))

aperes['APE'] = conditions

pdf = aperes[['date','APE','afternoon','morning','evening','peak_hr']]

pdf = pdf.set_index('date')

np.sum(pdf['APE'])

470

In [171]:
#hurdat removal of TC events
#replacing with gpdf

sgp = (36.6, -97.5)

tc_list = []

tc_dis_list = []

tc_time_list = []

for id_ in hurdat.keys:
    
    if int(id_[-4:]) < 2000:
        
        continue
    
    lat = np.array(hurdat.data[id_]['lat'][:])
    
    lon = np.array(hurdat.data[id_]['lon'][:])
    
    dis_list = []
    
    for j in range(len(lat)):
        
        pt = (lat[j],lon[j])
        
        dis_list = np.append(dis_list, haversine(sgp, pt, unit='km'))
    
    if np.any(dis_list < 800):
            
        tc_time_list = np.append(tc_time_list, np.array(hurdat.data[id_]['time'])[np.where(dis_list < 500)])
            
        tc_list.append(id_)


tc_time = []

for tt in tc_time_list:
    
    tc_time.append(np.datetime64(tt))   


ok_tc_time = []

for i in tc_time:
    
    ok_tc_time.append(i - pd.to_timedelta('6 hours'))
        
        
tc_dates = []

for dt in ok_tc_time:
    
    d = dt.date()
    
    tc_dates.append(d)


tc_dates = list(set(tc_dates))

pdf_minus_hu = pdf[~pdf.index.isin(tc_dates)]

pdf_only_hu = pdf[pdf.index.isin(tc_dates)]

print(pdf_minus_hu.shape)

print("ape count", sum(pdf_minus_hu['APE']==True))

(2749, 5)
ape count 467


In [172]:
with open('pdf.pdkl', 'wb') as file:
    
    pickle.dump(pdf, file)

## COMPARING TO GAOYUNS BELOW

In [31]:
with open('gpdf.pdkl', 'rb') as file:
    
    gpdf = pickle.load(file)

gpdf.shape

(2718, 6)

In [25]:
gpdf = gpdf.set_index('date')

gpdf.index = gpdf.index.date

apesgpdf = gpdf[gpdf['APE']==True]

KeyError: "None of ['date'] are in the columns"

In [23]:
apesgpdf.head()

NameError: name 'apesgpdf' is not defined

In [156]:
apespdf = pdf[pdf['APE']==True]

asindex = list(apespdf.index)

In [157]:
gsindex = list(apesgpdf.index)

In [158]:
missing = set(asindex) - set(gsindex)

In [164]:
len(missing)

26